In [1]:
import torch 
import torchtext 
import torch.nn as nn 
import torch.nn.functional as F 

import random
import math 
import time 

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
SEED = 1234 
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### Dataset

This task is to translate English to German

In [4]:
from datasets import load_dataset 

dataset = load_dataset("kvush/english_thai_texts")

c:\Users\araya\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_text', 'input_ids', 'translated_text', 'translated_ids', '__index_level_0__'],
        num_rows: 59859
    })
})

In [6]:
print(dataset["train"])

Dataset({
    features: ['input_text', 'input_ids', 'translated_text', 'translated_ids', '__index_level_0__'],
    num_rows: 59859
})


In [7]:
# train_data, valid_data, test_data = (dataset["train"], dataset["validation"], dataset["test"])

In [8]:
new_dataset = dataset.remove_columns(['input_ids', 'translated_ids', '__index_level_0__'])

In [9]:
new_dataset

DatasetDict({
    train: Dataset({
        features: ['input_text', 'translated_text'],
        num_rows: 59859
    })
})

In [10]:
train_data = new_dataset["train"]
train_data

Dataset({
    features: ['input_text', 'translated_text'],
    num_rows: 59859
})

In [11]:
print(train_data[0])
print(train_data[1])
print(train_data[2])
print(train_data[3])
print(train_data[4])

{'input_text': 'Service, scallops, all - top notch in every way!', 'translated_text': 'บริการหอยเชลล์ทั้งหมด - โดดเด่นในทุกด้าน!'}
{'input_text': 'Seth set the iron down on the table but it flipped over because the table was not durable.', 'translated_text': 'เสธวางเหล็กลงบนโต๊ะ แต่มันพลิกคว่ำเพราะโต๊ะไม่ทนทาน'}
{'input_text': 'Excellent food, wine and service!', 'translated_text': 'ยอดเยี่ยมกับอาหาร ไวน์ และบริการ!'}
{'input_text': "The best place in Vegas for gyros. I've had gyros in just about every place in town. Hands down this is the best.", 'translated_text': 'สถานที่ที่ดีที่สุดในเวกัสสำหรับไจโร ฉันได้ลองไจโรในเกือบทุกสถานที่ในเมืองนี้ และไม่มีข้อโต้แย้งใดๆ เลยว่านี่คือสิ่งที่ดีที่สุด'}
{'input_text': 'Kevin was worried Kenneth would win the cross country race, because Kevin was currently behind.', 'translated_text': 'เควินกังวลว่าเคนเน็ธจะชนะการแข่งขันครอสคันทรี เพราะเควินตามหลังอยู่'}


### Tokenizer

In [12]:
from pythainlp.tokenize import word_tokenize

text = "บริการหอยเชลล์ทั้งหมด - โดดเด่นในทุกด้าน!"

In [13]:
th_nlp = word_tokenize
th_nlp(text, engine="newmm")

['บริการ',
 'หอย',
 'เชลล์',
 'ทั้งหมด',
 ' ',
 '-',
 ' ',
 'โดดเด่น',
 'ใน',
 'ทุก',
 'ด้าน',
 '!']

In [14]:
import spacy
en_nlp = spacy.load("en_core_web_sm")
en_nlp

In [15]:
string = "Service, scallops, all - top notch in every way!"
[token.text for token in en_nlp.tokenizer(string)]

['Service',
 ',',
 'scallops',
 ',',
 'all',
 '-',
 'top',
 'notch',
 'in',
 'every',
 'way',
 '!']

In [16]:
# แปลงให้เป็น Token แล้วทำให้เป็นตัวเล็ก พร้อม ใส่ <sos> กับ <eos>
def tokenize_example(example, en_nlp, th_nlp, max_length, lower, sos_token, eos_token):
    en_tokens = [token.text for token in en_nlp.tokenizer(example["input_text"])][:max_length]
    th_tokens = th_nlp(example["translated_text"], engine="newmm")[:max_length]
    if lower:
        en_tokens = [token.lower() for token in en_tokens]
        # th_tokens = [token.lower() for token in th_tokens]
    en_tokens = [sos_token] + en_tokens + [eos_token]
    th_tokens = [sos_token] + th_tokens + [eos_token]
    return {"en_tokens" : en_tokens, "th_tokens" : th_tokens} # เพิ่ม list เข้าไป ซึ่งก็คือ en_tokens และ de_tokens

In [17]:
max_length = 1000
lower = True 
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "en_nlp" : en_nlp,
    "th_nlp" : th_nlp, 
    "max_length" : max_length,
    "lower" : lower,
    "sos_token" : sos_token,
    "eos_token" : eos_token
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)

In [18]:
train_data[0]

{'input_text': 'Service, scallops, all - top notch in every way!',
 'translated_text': 'บริการหอยเชลล์ทั้งหมด - โดดเด่นในทุกด้าน!',
 'en_tokens': ['<sos>',
  'service',
  ',',
  'scallops',
  ',',
  'all',
  '-',
  'top',
  'notch',
  'in',
  'every',
  'way',
  '!',
  '<eos>'],
 'th_tokens': ['<sos>',
  'บริการ',
  'หอย',
  'เชลล์',
  'ทั้งหมด',
  ' ',
  '-',
  ' ',
  'โดดเด่น',
  'ใน',
  'ทุก',
  'ด้าน',
  '!',
  '<eos>']}

In [19]:
# # แปลงให้เป็น Token แล้วทำให้เป็นตัวเล็ก พร้อม ใส่ <sos> กับ <eos>
# def thai_tokenize_example(sentence, th_nlp, max_length, sos_token, eos_token):
#     th_tokens = th_nlp(sentence, engine="newmm")[:max_length]
#     if lower:
#         pass
#         # th_tokens = [token.lower() for token in th_tokens]
#     th_tokens = [sos_token] + th_tokens + [eos_token]
#     return th_tokens # เพิ่ม list เข้าไป ซึ่งก็คือ en_tokens และ de_tokens

In [20]:
# max_length = 1000
# sos_token = "<sos>"
# eos_token = "<eos>"

# fn_kwargs = {
#     "th_nlp" : th_nlp, 
#     "max_length" : max_length,
#     "sos_token" : sos_token,
#     "eos_token" : eos_token
# }

# x = thai_tokenize_example("ฉันรักเธอ", th_nlp, max_length, sos_token, eos_token)

### Vocabularies

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F 
import random 
import numpy as np 
import spacy 
import datasets 
import torchtext
import tqdm 
import evaluate 


In [24]:
import tqdm 
import evaluate 

min_freq = 2 
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token
]

en_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["en_tokens"],
    min_freq=min_freq,
    specials=special_tokens
)

th_vocab = torchtext.vocab.build_vocab_from_iterator(
    train_data["th_tokens"],
    min_freq=min_freq,
    specials=special_tokens
)

In [32]:
en_vocab.get_itos()[:20]

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 '.',
 'the',
 ',',
 'to',
 'was',
 'and',
 'a',
 '!',
 'is',
 'i',
 'because',
 'for',
 'of',
 'but',
 'in',
 'it']

In [34]:
en_vocab.get_itos()[9]

'and'

In [33]:
th_vocab.get_itos()[:20]

['<unk>',
 '<pad>',
 '<sos>',
 '<eos>',
 ' ',
 'ที่',
 'และ',
 'ไม่',
 'ฉัน',
 'ของ',
 'มี',
 'ใน',
 'มาก',
 '!',
 'แต่',
 'เพราะ',
 'อาหาร',
 'ดี',
 'ได้',
 'เป็น']

In [37]:
en_vocab.get_stoi()["the"]

5

In [38]:
en_vocab["the"]

5

In [39]:
len(en_vocab), len(th_vocab)

(17029, 13363)

In [40]:
"the" in en_vocab

True

In [41]:
"The" in en_vocab

False

In [43]:
assert en_vocab[unk_token] == th_vocab[unk_token]
assert en_vocab[pad_token] == th_vocab[pad_token]

unk_index = en_vocab[unk_token]
pad_index = en_vocab[pad_token]



In [44]:
en_vocab.set_default_index(unk_index)
th_vocab.set_default_index(unk_index)

In [45]:
en_vocab["The"]

0

In [46]:
en_vocab.get_itos()[0]

'<unk>'

In [47]:
tokens = ["i", "love", "watching", "crime", "show"]

In [48]:
en_vocab.lookup_indices(tokens)

[13, 52, 896, 2687, 388]

In [49]:
en_vocab.lookup_tokens(en_vocab.lookup_indices(tokens))

['i', 'love', 'watching', 'crime', 'show']

In [50]:
def numericalize_example(example, en_vocab, th_vocab):
    en_ids = en_vocab.lookup_indices(example["en_tokens"])
    th_ids = th_vocab.lookup_indices(example["th_tokens"])
    return {"en_ids": en_ids, "th_ids": th_ids}

In [51]:
fn_kwargs = {"en_vocab": en_vocab, "th_vocab": th_vocab}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)

Map: 100%|██████████| 59859/59859 [00:08<00:00, 7093.04 examples/s]


In [52]:
train_data[0]

{'input_text': 'Service, scallops, all - top notch in every way!',
 'translated_text': 'บริการหอยเชลล์ทั้งหมด - โดดเด่นในทุกด้าน!',
 'en_tokens': ['<sos>',
  'service',
  ',',
  'scallops',
  ',',
  'all',
  '-',
  'top',
  'notch',
  'in',
  'every',
  'way',
  '!',
  '<eos>'],
 'th_tokens': ['<sos>',
  'บริการ',
  'หอย',
  'เชลล์',
  'ทั้งหมด',
  ' ',
  '-',
  ' ',
  'โดดเด่น',
  'ใน',
  'ทุก',
  'ด้าน',
  '!',
  '<eos>'],
 'en_ids': [2, 27, 6, 2518, 6, 65, 59, 453, 1368, 18, 199, 167, 11, 3],
 'th_ids': [2, 26, 1309, 2253, 171, 4, 143, 4, 624, 11, 313, 413, 13, 3]}

In [53]:
en_vocab.lookup_tokens(train_data[0]["en_ids"])

['<sos>',
 'service',
 ',',
 'scallops',
 ',',
 'all',
 '-',
 'top',
 'notch',
 'in',
 'every',
 'way',
 '!',
 '<eos>']

In [54]:
# สรุป คือ เราได้สร้าง dictionary แปลงเป็นตัวเลข(index) พร้อม mapไปกับdatasetแต่ละตัว

### เปลี่ยน index ให้เป็น Tensor

เราจะเปลี่ยน object ของ Train_data (ในที่นี้คือ en_ids และ de_ids) ไปเป็น Torch Tensor

In [55]:
data_type = "torch"
format_columns = ["en_ids", "th_ids"]

train_data = train_data.with_format(type=data_type, columns=format_columns, output_all_columns=True)

In [56]:
train_data[0]

{'en_ids': tensor([   2,   27,    6, 2518,    6,   65,   59,  453, 1368,   18,  199,  167,
           11,    3]),
 'th_ids': tensor([   2,   26, 1309, 2253,  171,    4,  143,    4,  624,   11,  313,  413,
           13,    3]),
 'input_text': 'Service, scallops, all - top notch in every way!',
 'translated_text': 'บริการหอยเชลล์ทั้งหมด - โดดเด่นในทุกด้าน!',
 'en_tokens': ['<sos>',
  'service',
  ',',
  'scallops',
  ',',
  'all',
  '-',
  'top',
  'notch',
  'in',
  'every',
  'way',
  '!',
  '<eos>'],
 'th_tokens': ['<sos>',
  'บริการ',
  'หอย',
  'เชลล์',
  'ทั้งหมด',
  ' ',
  '-',
  ' ',
  'โดดเด่น',
  'ใน',
  'ทุก',
  'ด้าน',
  '!',
  '<eos>']}

In [57]:
type(train_data[0]["en_ids"])

torch.Tensor

In [58]:
def get_collect_fn(pad_index):
    def collate_fn(batch):
        batch_en_ids = [example["en_ids"] for example in batch]
        batch_th_ids = [example["th_ids"] for example in batch]
        batch_en_ids = nn.utils.rnn.pad_sequence(batch_en_ids, padding_value=pad_index)
        batch_th_ids = nn.utils.rnn.pad_sequence(batch_th_ids, padding_value=pad_index)
        batch = {
            "en_ids" : batch_en_ids,
            "th_ids" : batch_th_ids
        }
        return batch 
    
    return collate_fn

In [59]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):
    collate_fn = get_collect_fn(pad_index)
    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle
    )
    return data_loader

In [60]:
batch_size = 128 
train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)

### Build the Model